In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_context("paper", font_scale=1.6)

def fasta_reader(file):
    '''Converts .fasta to a pandas dataframe with accession as index
    and sequence in a column 'sequence'
    '''
    fasta_df = pd.read_csv(file, sep='>', lineterminator='>', header=None)
    fasta_df[['Accession', 'Sequence']] = fasta_df[0].str.split('\n', 1, \
                                        expand=True)
    fasta_df['Accession'] = fasta_df['Accession']
    fasta_df['Sequence'] = fasta_df['Sequence'].replace('\n', '', regex=True).\
                            astype(str).str.upper().replace('U', 'T')
    total_seq = fasta_df.shape[0]
    fasta_df.drop(0, axis=1, inplace=True)
    fasta_df = fasta_df[fasta_df.Sequence != '']
    fasta_df = fasta_df[fasta_df.Sequence != 'NONE']
    final_df = fasta_df.dropna()
    remained_seq = final_df.shape[0]
    if total_seq != remained_seq:
        print("{} sequences were removed due to inconsistencies in"
                      "provided file.".format(total_seq-remained_seq))
    return final_df

In [7]:
rev_df_ = fasta_reader('../../Signal_Manuscript/Uniprot_all/Fungi/uniprot-reviewed yes+taxonomy 4751.fasta.gz')

rev_df_['Entry'] = rev_df_['Accession'].apply(lambda x: str(x).split('|')[1])
rev_df_ = rev_df_.rename(columns={'Sequence':'Protein'})
rev_df = rev_df_[~rev_df_.Protein.str[:75].str.contains('B|J|O|X|Z')].copy()

rev_ = rev_df[['Protein', 'Entry']].copy()


In [10]:
uniprot = pd.read_csv('../../Signal_Manuscript/Uniprot_all/Fungi/uniprot-reviewed yes+taxonomy 4751.tab.gz', sep='\t')
reviewed = uniprot.merge(rev_, on='Entry')
reviewed.shape

(34901, 37)

In [21]:
def predict(obj):
    res = obj.predict()
    try:
        f = obj.fungi()
    except Exception:
        f = ['Not a Signal peptide']
        
    try:
        t = obj.toxin()
    except Exception:
        t = ['Not a Signal peptide']

    return res +  f + t

def progress(iteration, total, message=None):
    '''Simple progressbar
    '''
    if message is None:
        message = ''
    bars_string = int(float(iteration) / float(total) * 50.)
    print("\r|%-50s| %d%% (%s/%s) %s "% ('█'*bars_string+ "░" * \
                                     (50 - bars_string), float(iteration)/\
                                     float(total) * 100, iteration, total, \
                                     message), end='\r', flush=True)

    if iteration == total:
        print('\nCompleted!')


from multiprocessing import Pool, cpu_count
import time
import warnings
warnings.filterwarnings("ignore")

from razor import RAZOR

In [ ]:
seqs_obj = [RAZOR(x, 60) for x in reviewed.Protein]

In [22]:



start_time = time.time()
pools = Pool(cpu_count()//4)
results = []
for res in pools.imap(predict, seqs_obj):
    results.append(res)
    progress(len(results), len(seqs_obj))
pools.close()
pools.join()
print("--- %s seconds --- for %d sequences.\n %f sequences per second"%(time.time() - start_time, reviewed.shape[0], \
                                             reviewed.shape[0]/(time.time() - start_time)))
print('\n Completed!')



reviewed['Preds'] = results
reviewed.to_pickle('uniprot_reviewed_scored_fungi.pkl.gz')

|██████████████████████████████████████████████████| 100% (34901/34901)  
Completed!
--- 125.8046190738678 seconds --- for 34901 sequences.
 277.422021 sequences per second

 Completed!


In [23]:
# Reviewed seqs

df = pd.read_pickle('uniprot_reviewed_scored_fungi.pkl.gz')
cols = ['SP', 'Y', 'C', 'All_Cleavage', 'Fungi_Preds', 'Fungi_prob', 'Toxin_Preds', 'Toxin_prob']

tox_fung = df[df['Preds'].apply(len) == 8].copy()

tox_fung[cols] = pd.DataFrame(tox_fung.Preds.tolist(), index= tox_fung.index)


In [24]:
#Take those with 5 models returning True for toxins
probable_sp = tox_fung[tox_fung['SP'].apply(lambda x: (x == True).sum()) == 5].copy()

probable_fungi = probable_sp[probable_sp['Fungi_Preds'].apply(lambda x: (x ==True).sum()) == 5].copy()
print('Reviewed set has {} probable fungi.'.format(probable_fungi.shape[0]))
# print('====\n')
# print(','.join([i for i in probable_fungi.Entry.values]))
# print('====\n')

Reviewed set has 473 probable fungi.


In [25]:
# See if any from training is in the predicted sequences
# and remove them

train = pd.read_pickle('../results/fungi_nonfungi.pkl.gz')
train = train[train.Label == 1]

all_train_atp_seq = set(list(train.Entry.values))

In [26]:
train.shape

(121, 42)

In [27]:
# filtered__ = df_uniprot[df_uniprot['Entry'].apply(lambda x: x not in all_train_atp_seq)].copy()
filtered__ = probable_fungi[probable_fungi['Entry'].apply(lambda x: x not in all_train_atp_seq)].copy()
filtered__.head(2)

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Signal peptide,Taxonomic lineage IDs,Date of creation,...,Protein,Preds,SP,Y,C,All_Cleavage,Fungi_Preds,Fungi_prob,Toxin_Preds,Toxin_prob
9,D4ARV2,AIM6_ARTBC,reviewed,Altered inheritance of mitochondria protein 6 ...,ARB_06966,Arthroderma benhamiae (strain ATCC MYA-4681 / ...,314,"SIGNAL 1..21; /evidence=""ECO:0000255""",663331,2015-10-14,...,MKSSILASAAILAASLEPVAALSEPLQNILINTDRSPIYRYPTDVT...,"[[True, True, True, True, True], 0.73498299300...","[True, True, True, True, True]",0.734983,0.74,"[21, 21]","[True, True, True, True, True]","[0.53, 0.37, 0.43, 0.32, 0.46]","[False, False, False, False, False]","[0.15, 0.11, 0.12, 0.1, 0.16]"
66,Q0CF74,AZPB3_ASPTN,reviewed,FAD-linked oxidoreductase ATEG_07660 (EC 1.-.-...,ATEG_07660,Aspergillus terreus (strain NIH 2624 / FGSC A1...,481,"SIGNAL 1..17; /evidence=""ECO:0000255""",341663,2020-06-17,...,MLRQLYQSLALVAAVHATGIFANGVDIESLFGPYLSPGTEIAEPTD...,"[[True, True, True, True, True], 0.65482822174...","[True, True, True, True, True]",0.654828,0.71,[22],"[True, True, True, True, True]","[0.24, 0.26, 0.37, 0.24, 0.26]","[False, False, False, False, False]","[0.04, 0.08, 0.06, 0.05, 0.07]"


In [28]:
def find_tm(annot):
    '''
    Find transmembrane annotation if it is outside 
    first 50 residues
    '''
    annot = str(annot)
    if annot != 'NaN':
        try:
            if int(annot.split(';')[0].split(' ')[1].split('..')[0]) < 50:
                return True
            else:
                return False
        except Exception:
#             print(annot)
            return False
    else:
        return False

In [31]:

sp_fungi = filtered__[~filtered__['Transmembrane'].apply(find_tm)].copy()

sp_fungi[sp_fungi['Signal peptide'].isna()]

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Signal peptide,Taxonomic lineage IDs,Date of creation,...,Protein,Preds,SP,Y,C,All_Cleavage,Fungi_Preds,Fungi_prob,Toxin_Preds,Toxin_prob
264,Q01302,CCG6_NEUCR,reviewed,Clock-controlled protein 6,ccg-6 B8P8.100 NCU01418,Neurospora crassa (strain ATCC 24698 / 74-OR23...,142,NaN,367110,1997-11-01,...,MKFSAAAVLAAAAGAHAWSNVTYTTEIVTAVTTYCPGPTEITHGGN...,"[[True, True, True, True, True], 0.71847059786...","[True, True, True, True, True]",0.718471,0.86,[17],"[True, True, True, True, True]","[0.34, 0.54, 0.43, 0.35, 0.31]","[False, False, False, False, False]","[0.1, 0.11, 0.09, 0.08, 0.09]"
614,P87167,BUT2_SCHPO,reviewed,Uba3-binding protein but2,but2 SPBC3D6.02,Schizosaccharomyces pombe (strain 972 / ATCC 2...,390,NaN,284812,1998-07-15,...,MQLLNSFLGFAASIAVLASSADAAPTLYKRKSKSNTNTAKSVAILD...,"[[True, True, True, True, True], 0.74458041875...","[True, True, True, True, True]",0.744580,0.68,[23],"[True, True, True, True, True]","[0.48, 0.27, 0.45, 0.5, 0.54]","[False, False, False, False, False]","[0.06, 0.11, 0.1, 0.06, 0.12]"
7939,G2TRT4,NEW25_SCHPO,reviewed,SAP domain-containing new25,new25 SPCC330.21,Schizosaccharomyces pombe (strain 972 / ATCC 2...,89,NaN,284812,2012-04-18,...,MKIANVLTLSFAAIIAASSLAVVVSPITDNSRETASSSISQSPPSQ...,"[[True, True, True, True, True], 0.65268675488...","[True, True, True, True, True]",0.652687,0.71,[21],"[True, True, True, True, True]","[0.68, 0.58, 0.49, 0.68, 0.49]","[False, False, False, False, False]","[0.15, 0.21, 0.13, 0.22, 0.14]"
31525,A0A0B4GDU5,ARP1_METBS,reviewed,Scytalone dehydratase-like protein Arp1 (EC 4....,Arp1 MBR_03976,Metarhizium brunneum (strain ARSEF 3297),744,NaN,1276141,2018-12-05,...,MGWNQTFLFLAFAATAASSLVGSGTSLQLNGIDYFVSPFSQGKVTN...,"[[True, True, True, True, True], 0.83426614458...","[True, True, True, True, True]",0.834266,0.84,"[19, 19]","[True, True, True, True, True]","[0.39, 0.4, 0.34, 0.28, 0.26]","[False, False, False, False, False]","[0.03, 0.04, 0.05, 0.07, 0.01]"


In [32]:
new_identified = sp_fungi[sp_fungi['Signal peptide'].isna()].copy()
new_identified['Entry_'] = '>' + new_identified['Entry']
for i, v in new_identified.Entry_.items():
    print("{}\n{}".format(v, new_identified.Protein[i]))

>Q01302
MKFSAAAVLAAAAGAHAWSNVTYTTEIVTAVTTYCPGPTEITHGGNTYTVTEATTLTISDCPCTVTKPIITTSSVICHSCTGYVNSTIPAPTSAGSVGTGSAPAVVTPTVSPSEVPTAGAGKAAALSGAGLVGVLGLAAILL
>P87167
MQLLNSFLGFAASIAVLASSADAAPTLYKRKSKSNTNTAKSVAILDGVGVNNTFGVISYREKFPMDYHIWHTAPSTGKTYLQPWNDAVEPSTYYIDEGTFLRTGLKFAYIGDNMDLAFTNHTDWKASKHDDGTHRVDLSQRKPANNFMATQRCGKLDPFGYQLKRSKKVFQACGTQVFVGSGRSIDCQWMDSVALNLSRYYGNTEALSSLPLPRNLSADIPAKSSRLFPHGIYNFNFSAPNKRMQRFTASEATTVNGQNQSVYLTYDVPYGRSLYYYTSCALEFRFTNEFFPMDVTPNDGSAQFVVYNMSGNPKKQTTSSKGPTRLYEVARFNCTTRGCEYTQNIPCPRAGHSHTYELAPASPNTSISWIQSYSPRIGVTLQVYSNANFD
>G2TRT4
MKIANVLTLSFAAIIAASSLAVVVSPITDNSRETASSSISQSPPSQWSKKQLIEYCKKNSLKTSGSHEELVIRVQNHLRTASKKVDARP
>A0A0B4GDU5
MGWNQTFLFLAFAATAASSLVGSGTSLQLNGIDYFVSPFSQGKVTNGSVAINTRQNQLGFVPATVIAGDLYTESTLQSLFLNWSTVDDVWQPAFLETIFVFNFAKLTNKNHNYHDGVSSSVFPLQVTHKIPSGPYFLNVHTGEVHPAYRLYDDFAGAFTQSLLQRPDGRFQTLSAQVPAAASITIGVPSRLYFTKTEAKPLAGVRIGVKDIFSLAGVKKGCGNRAWYHLYPVANSTGTAMQNLIDKGAIIVGVQKTSQFANGETPTADWVDYHSPFNPRGDGYQDPATSSAGAGSSIASYEWLDLAVGSDTGGSIRGPATVQGIFGNRPSHGLVSLDNVM

In [33]:
probable_fungi[probable_fungi.Entry.apply(lambda x: x in new_identified.Entry.values)][['Entry', 'All_Cleavage', 'Y']]

,Entry,All_Cleavage,Y
264,Q01302,[17],0.718471
614,P87167,[23],0.744580
7939,G2TRT4,[21],0.652687
31525,A0A0B4GDU5,"[19, 19]",0.834266


### Submitted to SignalP 5.0

In [37]:
signalp5 = {"INFO":{"failedjobs":0,"size":4},"CSV_FILE":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_protein_type.txt","MATURE_FILE":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_mature.fasta","GFF_FILE":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output.gff3","ZIP_FILE":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_all_results.zip","SEQUENCES":{"A0A0B4GDU5":{"CS_pos":"Cleavagesitebetweenpos.19and20:ASS-LV.Probability:0.1952","Likelihood":[0.9401,0.0599],"Name":"A0A0B4GDU5","Plot_eps":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_A0A0B4GDU5_plot.eps","Plot_png":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_A0A0B4GDU5_plot.png","Plot_txt":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_A0A0B4GDU5_pred.txt","Prediction":"Signalpeptide(Sec/SPI)","Protein_types":["SignalPeptide(Sec/SPI)","Other"]},"G2TRT4":{"CS_pos":"Cleavagesitebetweenpos.21and22:SLA-VV.Probability:0.6180","Likelihood":[0.9506,0.0494],"Name":"G2TRT4","Plot_eps":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_G2TRT4_plot.eps","Plot_png":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_G2TRT4_plot.png","Plot_txt":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_G2TRT4_pred.txt","Prediction":"Signalpeptide(Sec/SPI)","Protein_types":["SignalPeptide(Sec/SPI)","Other"]},"P87167":{"CS_pos":"Cleavagesitebetweenpos.23and24:ADA-AP.Probability:0.8428","Likelihood":[0.9721,0.0279],"Name":"P87167","Plot_eps":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_P87167_plot.eps","Plot_png":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_P87167_plot.png","Plot_txt":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_P87167_pred.txt","Prediction":"Signalpeptide(Sec/SPI)","Protein_types":["SignalPeptide(Sec/SPI)","Other"]},"Q01302":{"CS_pos":"Cleavagesitebetweenpos.17and18:AHA-WS.Probability:0.6092","Likelihood":[0.9975,0.0025],"Name":"Q01302","Plot_eps":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_Q01302_plot.eps","Plot_png":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_Q01302_plot.png","Plot_txt":"/services/SignalP-5.0/tmp/5FB19EFE0000792FBC685427/output_Q01302_pred.txt","Prediction":"Signalpeptide(Sec/SPI)","Protein_types":["SignalPeptide(Sec/SPI)","Other"]}},"FORMAT":"long","ORG":"Eukarya"}

In [38]:
sp5 = pd.DataFrame(signalp5)
sp5

,INFO,CSV_FILE,MATURE_FILE,GFF_FILE,ZIP_FILE,SEQUENCES,FORMAT,ORG
failedjobs,0.0,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,NaN,long,Eukarya
size,4.0,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,NaN,long,Eukarya
A0A0B4GDU5,NaN,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,{'CS_pos': 'Cleavagesitebetweenpos.19and20:ASS...,long,Eukarya
G2TRT4,NaN,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,{'CS_pos': 'Cleavagesitebetweenpos.21and22:SLA...,long,Eukarya
P87167,NaN,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,{'CS_pos': 'Cleavagesitebetweenpos.23and24:ADA...,long,Eukarya
Q01302,NaN,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,/services/SignalP-5.0/tmp/5FB19EFE0000792FBC68...,{'CS_pos': 'Cleavagesitebetweenpos.17and18:AHA...,long,Eukarya


In [39]:
sp5[~sp5['SEQUENCES'].isna()]['SEQUENCES']

A0A0B4GDU5    {'CS_pos': 'Cleavagesitebetweenpos.19and20:ASS...
G2TRT4        {'CS_pos': 'Cleavagesitebetweenpos.21and22:SLA...
P87167        {'CS_pos': 'Cleavagesitebetweenpos.23and24:ADA...
Q01302        {'CS_pos': 'Cleavagesitebetweenpos.17and18:AHA...
Name: SEQUENCES, dtype: object

### Deepsig

deepsig prediction results
  Copyright (C) 2017 Castrense Savojardo
  Department of Biological, Geological and Environmental Sciences (BiGeA)
  Bologna Biocomputing Group
  University of Bologna, Italy.
  savojard@biocomp.unibo.it



#### ProteinID	PredictedSignal	Reliability	CleavageSite (if any)
---------------------------------------------------------------------

Q01302	SignalPeptide	0.75	17

P87167	SignalPeptide	0.99	23

G2TRT4	SignalPeptide	0.96	21

A0A0B4GDU5	SignalPeptide	0.99	18



In [41]:
final_results = pd.DataFrame({'Entry':['A0A0B4GDU5', 'G2TRT4', 'P87167', 'Q01302'],\
                             'Razor (SP (Cleavage))':['Yes (19)', 'Yes (21)', 'Yes (23)', 'Yes (17)'],\
                             'SignalP 5.0 (SP (Cleavage))': ['Yes (19)', 'Yes (21)', 'Yes (23)', 'Yes (17)'],\
                             'DeepSig (SP (Cleavage))': ['Yes (18)', 'Yes(21)', 'Yes (23)', 'Yes (17)']})
final_results

,Entry,Razor (SP (Cleavage)),SignalP 5.0 (SP (Cleavage)),DeepSig (SP (Cleavage))
0,A0A0B4GDU5,Yes (19),Yes (19),Yes (18)
1,G2TRT4,Yes (21),Yes (21),Yes(21)
2,P87167,Yes (23),Yes (23),Yes (23)
3,Q01302,Yes (17),Yes (17),Yes (17)
